In [ ]:
import re
import pandas as pd
import numpy as np
import os
import tqdm


In [ ]:
def stackDates(df, value_name, year_col_name='Year', month_col_name='Mo'):
	df['yyyy-mm'] = df[year_col_name]+'-'+df[month_col_name].apply(lambda x: str(x).zfill(2))
	df.drop(columns=[year_col_name, month_col_name], inplace=True)
	df.set_index('yyyy-mm', inplace=True)
	stacked_df = df.stack().reset_index().rename(columns={'level_1': 'day', 0: value_name})
	stacked_df['yyyy-mm-dd'] = stacked_df['yyyy-mm'] + '-' + stacked_df['day']
	stacked_df.drop(columns=['yyyy-mm', 'day'], inplace=True)
	stacked_df.set_index('yyyy-mm-dd', inplace=True)
	return stacked_df
def readTempTextFile(path):   
    df_list = []
    with open(path) as f:
        lines = f.readlines()
        lines[2] = lines[2].replace('Day ', '')
        details = lines[0].split(',')
        details = [item.strip() for item in details]
        for line in lines[3:]:
            if 'Jour' in line:
                continue
            # remove whitespace at the start and the newline at the end
            line = line.strip()
            # split each column on whitespace
            columns = re.split('\s+', line, maxsplit=35)
            df_list.append(columns)
        headers=re.split('\s+', lines[2])[1:-1]
    f.close
    return pd.DataFrame(df_list, columns=headers), details

In [ ]:

folder_name = 'Homog_daily_mean_temp_v2022_Gen3'
folder_path = '../data/' + folder_name + '/'
output_folder = '../results/' + folder_name + '/'
os.makedirs(output_folder, exist_ok=True)
for filename in os.listdir(folder_path):
    input_path = folder_path + filename
    temp_df, details = readTempTextFile(input_path)
    value_name = details[4].title() + ' [' + details[5] + ']'
    print(input_path)
    output_path = output_folder + filename[:-4] +'.csv'
    output_df = stackDates(temp_df, value_name)
    output_df.to_csv(output_path)
    mask = output_df.apply(lambda row: any(['M' in val for val in row.values]), axis=1)
    display(output_df[mask])
    break
    
